In [2]:
!pip install numpy==1.26.4 --quiet

In [1]:
!pip install -q torch sentence-transformers faiss-cpu fasttext tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.1

In [2]:
# --- Colab setup (optional but recommended) ---
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, json, re, shutil, traceback, hashlib, time
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import fasttext
import torch

# =========================
# CONFIG
# =========================
DATA_DIR   = '/content/drive/MyDrive/Nour/users_posts_json'
OUTPUT_DIR = '/content/drive/MyDrive/Nour/bluesky/full_index'

# Sharding
SHARD_MAX_VECTORS = 1_500_000  # ~2.3 GB per shard with 384-d float32
MANIFEST_PATH     = os.path.join(OUTPUT_DIR, "index_manifest.json")

# Process log (global, across all shards)
PROCESSED_PATH    = os.path.join(OUTPUT_DIR, 'processed_files.txt')

# Language ID cache
LID_PATH          = os.path.join(OUTPUT_DIR, 'lid.176.bin')

# Batching
BATCH_SIZE        = 5000

os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# UTIL: manifest + shard paths
# =========================
def load_manifest():
    if os.path.exists(MANIFEST_PATH):
        with open(MANIFEST_PATH, "r") as f:
            return json.load(f)
    # fresh start
    return {"current_shard": 1, "shards": []}

def save_manifest(m):
    tmp = MANIFEST_PATH + ".tmp"
    with open(tmp, "w") as f:
        json.dump(m, f)
    os.replace(tmp, MANIFEST_PATH)

def shard_paths(shard_id: int):
    base = os.path.join(OUTPUT_DIR, f"all_posts_index_shard_{shard_id:04d}.faiss")
    return base, base + ".tmp", base + ".bak"

# =========================
# DOWNLOAD/CACHE fastText LID MODEL
# =========================
if not os.path.exists(LID_PATH):
    # Colab shell magic is fine here
    !wget -q https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin -O "$LID_PATH"

# =========================
# LOAD MODELS
# =========================
print("Loading fastText LID...")
ft_model = fasttext.load_model(LID_PATH)

use_gpu = torch.cuda.is_available()
device  = 'cuda' if use_gpu else 'cpu'
print(f"Loading sentence-transformer on {device}...")
model = SentenceTransformer('all-MiniLM-L12-v2', device=device)
EMB_DIM = model.get_sentence_embedding_dimension()  # typically 384

# speed tweaks
if use_gpu:
    try:
        model.auto_model.half()
    except Exception:
        pass
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
os.environ["TOKENIZERS_PARALLELISM"] = "false"
if use_gpu:
    torch.set_num_threads(1)

# =========================
# HELPERS
# =========================
def is_english(text, threshold=0.5):
    if not text or not text.strip():
        return False
    text = text.strip().replace('\n', ' ')
    latin_chars = len(re.findall(r'[a-zA-Z]', text))
    total_chars = len(text)
    if total_chars == 0 or (latin_chars / total_chars) < threshold:
        return False
    label = ft_model.predict(text)[0][0]
    return label == '__label__en'

def clean_text(text: str):
    text = text.lower().strip()
    text = re.sub(r'https?://\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text if len(text.split()) >= 3 else None

def atomic_write(src_path, dst_path, bak_path=None):
    # rotate backup
    if bak_path and os.path.exists(dst_path):
        shutil.copy2(dst_path, bak_path)
    # atomic replace
    os.replace(src_path, dst_path)

def uri_to_int64(uri: str) -> np.int64:
    h = hashlib.blake2b(uri.encode('utf-8'), digest_size=8).digest()
    return np.frombuffer(h, dtype=np.uint64)[0].view(np.int64)

def pick_bs():
    if torch.cuda.is_available():
        name = torch.cuda.get_device_name(0).lower()
        if 'a100' in name: return 4096
        if 'v100' in name: return 2048
        if 't4'   in name: return 1024
    return 256

AUTO_BS = pick_bs()

def encode_batch(texts, bs=None):
    if bs is None: bs = AUTO_BS
    return model.encode(
        texts,
        batch_size=bs,
        convert_to_numpy=True,
        normalize_embeddings=True,  # unit vectors; IP == cosine
        show_progress_bar=False
    )

def new_index():
    base = faiss.IndexFlatIP(EMB_DIM)
    return faiss.IndexIDMap2(base)

def get_existing_ids(ix):
    # robustly extract IDs from IndexIDMap2 across FAISS builds
    try:
        arr = faiss.vector_to_array(ix.id_map)
        return set(map(int, arr))
    except Exception:
        try:
            imap = faiss.downcast_IndexIDMap(ix).id_map
            arr = faiss.vector_to_array(imap)
            return set(map(int, arr))
        except Exception:
            return set()

# =========================
# INIT / RESUME SHARD
# =========================
manifest = load_manifest()
shard_id = manifest["current_shard"]
INDEX_PATH, INDEX_TMP, INDEX_BAK = shard_paths(shard_id)

try:
    if os.path.exists(INDEX_PATH):
        index = faiss.read_index(INDEX_PATH)
        if not isinstance(index, faiss.IndexIDMap2):
            index = faiss.IndexIDMap2(index)
        print(f"Resuming shard {shard_id:04d}")
    elif os.path.exists(INDEX_BAK):
        index = faiss.read_index(INDEX_BAK)
        if not isinstance(index, faiss.IndexIDMap2):
            index = faiss.IndexIDMap2(index)
        print(f"Resuming shard {shard_id:04d} from backup")
    else:
        index = new_index()
        print(f"Starting new shard {shard_id:04d}")
except Exception as e:
    print(f"Failed to load shard {shard_id:04d}: {e}")
    index = new_index()

existing_ids_shard = get_existing_ids(index)
print(f"[SHARD {shard_id:04d}] vectors={index.ntotal}, unique_ids_in_shard={len(existing_ids_shard)}")

# =========================
# LOAD processed_files (global)
# =========================
if os.path.exists(PROCESSED_PATH):
    with open(PROCESSED_PATH, 'r') as f:
        processed_files = set(line.strip() for line in f if line.strip())
else:
    processed_files = set()

# =========================
# MAIN LOOP
# =========================
user_files    = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.json')])
pending_files = [f for f in user_files if f not in processed_files]

print("Resume status:")
print(f"- Files total:       {len(user_files)}")
print(f"- Already processed: {len(processed_files)}")
print(f"- Pending now:       {len(pending_files)}")

batch_texts, batch_uris = [], []

def process_batch():
    """Encode, add with IDs, checkpoint to current shard (dedup inside shard)."""
    global batch_texts, batch_uris, index, existing_ids_shard
    if not batch_texts:
        return
    try:
        # map URIs -> int64
        ids64 = [int(uri_to_int64(u)) for u in batch_uris]
        # drop any IDs already present in this shard
        keep_mask = [i not in existing_ids_shard for i in ids64]
        if not any(keep_mask):
            batch_texts.clear(); batch_uris.clear(); return

        texts_kept = [t for t,k in zip(batch_texts, keep_mask) if k]
        ids_kept   = np.array([i for i,k in zip(ids64, keep_mask) if k], dtype='int64')

        embeddings = encode_batch(texts_kept)
        before = index.ntotal
        index.add_with_ids(embeddings, ids_kept)
        added = index.ntotal - before
        # update in-memory dedupe for this shard
        existing_ids_shard.update(map(int, ids_kept))

        # checkpoint current shard
        faiss.write_index(index, INDEX_TMP)
        atomic_write(INDEX_TMP, INDEX_PATH, INDEX_BAK)

        print(f"[SHARD {shard_id:04d}] Checkpointed: requested {len(ids_kept)} | actually added {added} | Total={index.ntotal}")
    except Exception as e:
        print(f"[ERROR] Failed to process batch: {e}")
        traceback.print_exc()
    finally:
        batch_texts.clear()
        batch_uris.clear()

# progress bar seeded with already-completed count (for visibility only)
pbar = tqdm(total=len(user_files), initial=len(processed_files))
for fname in user_files:
    if fname in processed_files:
        pbar.update(1)
        continue

    file_path = os.path.join(DATA_DIR, fname)
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"[ERROR] Failed to read {fname}: {e}")
        # don't mark as processed; try again next run
        pbar.update(1)
        continue

    # ingest posts
    for entry in data:
        post   = entry.get("post")   or {}
        record = post.get("record")  or {}
        text   = record.get("text")  or ""
        if not text: continue
        if not is_english(text): continue
        cleaned = clean_text(text)
        if not cleaned: continue

        uri = post.get("uri")
        if not uri: continue

        batch_texts.append(cleaned)
        batch_uris.append(uri)

        if len(batch_texts) >= BATCH_SIZE:
            process_batch()

    # flush whatever’s left from this file BEFORE marking file done
    process_batch()

    # mark file processed (append-only + fsync)
    with open(PROCESSED_PATH, 'a') as f:
        f.write(fname + '\n')
        f.flush()
        os.fsync(f.fileno())
    processed_files.add(fname)
    pbar.update(1)

    # ROLL OVER to a new shard ONLY BETWEEN FILES
    if index.ntotal >= SHARD_MAX_VECTORS:
        # Seal this shard to disk
        faiss.write_index(index, INDEX_TMP)
        os.replace(INDEX_TMP, INDEX_PATH)

        # Record sealed shard in manifest (only once)
        already = any(s.get("id") == shard_id for s in manifest["shards"])
        if not already:
            manifest["shards"].append({
                "id": shard_id,
                "path": INDEX_PATH,
                "vectors": int(index.ntotal)
            })

        # Start a new shard
        shard_id += 1
        manifest["current_shard"] = shard_id
        save_manifest(manifest)

        # Reset index + paths + dedupe set
        INDEX_PATH, INDEX_TMP, INDEX_BAK = shard_paths(shard_id)
        index = new_index()
        existing_ids_shard = set()
        print(f"Rolled over to new shard {shard_id:04d}")

pbar.close()

# final safety flush (usually empty)
process_batch()

# Ensure manifest is up to date for the current open shard (not sealed yet)
# (We don't add current shard to manifest["shards"] until it is sealed by rollover)
save_manifest(manifest)

print(f"\nDONE. Current shard {shard_id:04d} vectors: {index.ntotal}")
print(f"Shards sealed: {len(manifest['shards'])}")
for s in manifest["shards"]:
    print(f" - shard {s['id']:04d}: {s['vectors']} vectors -> {s['path']}")


Mounted at /content/drive
Loading fastText LID...
Loading sentence-transformer on cuda...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

[SHARD 0019] Checkpointed: requested 2313 | actually added 2313 | Total=231650


11074it [2:52:05,  6.31s/it]

[SHARD 0019] Checkpointed: requested 9 | actually added 9 | Total=231659
[SHARD 0019] Checkpointed: requested 4920 | actually added 4920 | Total=236579
[SHARD 0019] Checkpointed: requested 4855 | actually added 4855 | Total=241434
[SHARD 0019] Checkpointed: requested 4965 | actually added 4965 | Total=246399


11075it [2:52:31,  1.79s/it]

[SHARD 0019] Checkpointed: requested 282 | actually added 282 | Total=246681

DONE. Current shard 0019 vectors: 246681
Shards sealed: 18
 - shard 0001: 1523376 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0001.faiss
 - shard 0002: 1501978 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0002.faiss
 - shard 0003: 1507652 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0003.faiss
 - shard 0004: 1503669 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0004.faiss
 - shard 0005: 1547739 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0005.faiss
 - shard 0006: 1501713 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0006.faiss
 - shard 0007: 1503075 vectors -> /content/drive/MyDrive/Nour/bluesky/full_index/all_posts_index_shard_0007.faiss
 - shard 0008: 1509695 vectors -> /content/drive/MyDrive/Nour/blu